# Import Libraries

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import numpy as numpy

# Review & Rating Scraping

In [16]:
def get_product_urls(url_main):
    product_urls = []
    driver = webdriver.Chrome()

    try:
        driver.get(url_main)
        sleep(2)

        # Scroll the page to load all links
        for _ in range(15):  # Scroll n times
            driver.execute_script("window.scrollBy(0, 250)")
            sleep(1)

        # Get the page source after scrolling
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Find all divs with the class containing product information
        product_divs = soup.find_all('div', {"class": "css-1asz3by"})

        # Extract href from nested <a> tags inside the divs
        for div in product_divs:
            a_tag = div.find('a')  # Locate the <a> tag inside the div
            if a_tag and a_tag.get('href'):
                product_urls.append(a_tag['href'])

    finally:
        driver.quit()

    return product_urls

In [17]:
def scrape_reviews_and_ratings(product_urls):
    reviews = []
    ratings = []

    driver = webdriver.Chrome()

    try:
        for product_url in product_urls:
            print(f"Processing: {product_url}")
            driver.get(product_url)

            while True:
                for _ in range(20):
                    driver.execute_script("window.scrollBy(0, 250)")
                    sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")

                for product in soup.find_all('div', {"class": "css-1k41fl7"}):
                    review_element = product.find('span', {"data-testid": "lblItemUlasan"})
                    reviews.append(review_element.get_text() if review_element else 'None')

                    rating_element = product.find('div', {"class": "rating"})
                    ratings.append(rating_element.get('aria-label') if rating_element else 'None')

                try:
                    next_button_container = driver.find_element(By.CLASS_NAME, "css-1xqkwi8")
                    next_button = next_button_container.find_element(
                        By.XPATH, './/button[contains(@class, "css-16uzo3v-unf-pagination-item") and @aria-label="Laman berikutnya"]'
                    )
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    driver.execute_script("arguments[0].click();", next_button)
                    sleep(2)
                except (NoSuchElementException, TimeoutException):
                    print("No more pages to navigate for this product.")
                    break

    finally:
        driver.quit()

    print(f"Scraped {len(reviews)} reviews and {len(ratings)} ratings.")  # Debugging output
    data = pd.DataFrame({'Review': reviews, 'Rating': ratings})
    return data

In [18]:
def main(url_main):
    urls = []

    for url in url_main:
        # Get the product URLs from the main page
        product_urls = get_product_urls(url)
        for urll in product_urls:    
            urls.append(urll)

    # Scrape reviews and ratings for each product
    data = scrape_reviews_and_ratings(urls)

    # Optionally, save the data to a CSV file
    data.to_csv("reviews.csv", index=False)
    return data

In [ ]:
urlss = ['https://www.tokopedia.com/samsung/etalase/mobiles',
       'https://www.tokopedia.com/xiaomi/etalase/mobile']

urls = []

for url in urlss:
    # Get the product URLs from the main page
    product_urls = get_product_urls(url)
    for urll in product_urls:    
        urls.append(urll)

reviews = []
ratings = []

driver = webdriver.Chrome()

try:
    for product_url in urls:
        print(f"Processing: {product_url}")
        driver.get(product_url)

        while True:
            for _ in range(20):
                driver.execute_script("window.scrollBy(0, 250)")
                sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            for product in soup.find_all('div', {"class": "css-1k41fl7"}):
                review_element = product.find('span', {"data-testid": "lblItemUlasan"})
                reviews.append(review_element.get_text() if review_element else 'None')

                rating_element = product.find('div', {"class": "rating"})
                ratings.append(rating_element.get('aria-label') if rating_element else 'None')

            try:
                next_button_container = driver.find_element(By.CLASS_NAME, "css-1xqkwi8")
                next_button = next_button_container.find_element(
                    By.XPATH, './/button[contains(@class, "css-16uzo3v-unf-pagination-item") and @aria-label="Laman berikutnya"]'
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                driver.execute_script("arguments[0].click();", next_button)
                sleep(2)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to navigate for this product.")
                break

finally:
    driver.quit()

print(f"Scraped {len(reviews)} reviews and {len(ratings)} ratings.")  # Debugging output
data = pd.DataFrame({'Review': reviews, 'Rating': ratings})

Processing: https://www.tokopedia.com/xiaomi/official-poco-x6-pro-5g-dimensity-8300-ultra-120hz-fiow-amoled-67w-t-grey-12-512g-ed3b0?extParam=src%3Dshop%26whid%3D13418340
No more pages to navigate for this product.
Processing: https://www.tokopedia.com/xiaomi/official-poco-c65-ram-hingga-16-gb-prosesor-kencang-helio-g85-90hz-black-8-256g-5d9be?extParam=src%3Dshop%26whid%3D13418340


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF72EF5AD02+56930]
	(No symbol) [0x00007FF72EECF602]
	(No symbol) [0x00007FF72ED842E5]
	(No symbol) [0x00007FF72ED71FDF]
	(No symbol) [0x00007FF72ED71E11]
	(No symbol) [0x00007FF72ED70625]
	(No symbol) [0x00007FF72ED710AF]
	(No symbol) [0x00007FF72ED7DF3E]
	(No symbol) [0x00007FF72ED9009E]
	(No symbol) [0x00007FF72ED94E5A]
	(No symbol) [0x00007FF72ED71600]
	(No symbol) [0x00007FF72ED8FECD]
	(No symbol) [0x00007FF72EE080BD]
	(No symbol) [0x00007FF72EDEBA43]
	(No symbol) [0x00007FF72EDBD438]
	(No symbol) [0x00007FF72EDBE4D1]
	GetHandleVerifier [0x00007FF72F2D6F8D+3711213]
	GetHandleVerifier [0x00007FF72F3304CD+4077101]
	GetHandleVerifier [0x00007FF72F32865F+4044735]
	GetHandleVerifier [0x00007FF72EFF9736+706710]
	(No symbol) [0x00007FF72EEDB8DF]
	(No symbol) [0x00007FF72EED6AC4]
	(No symbol) [0x00007FF72EED6C1C]
	(No symbol)

In [20]:
print(f"Scraped {len(reviews)} reviews and {len(ratings)} ratings.")  # Debugging output
data = pd.DataFrame({'Review': reviews, 'Rating': ratings})

Scraped 3800 reviews and 3800 ratings.


In [ ]:
data.to_csv("reviews_3.csv", index=False)

In [ ]:
data1 = pd.read_csv('product_reviews.csv')
data2 = pd.read_csv('reviews_1.csv')
data3 = pd.read_csv('reviews_2.csv')
data4 = pd.read_csv('reviews_3.csv')

data5 = pd.concat([data1, data2, data3, data4])

data5.to_csv('data_final.csv')

# Product Scraping

In [ ]:
driver = webdriver.Chrome()

names, prices, ratings_overall, solds= [], [], [], []

url = 'https://www.tokopedia.com/officialinfinix/etalase/note-series'
driver.get(url)

# Scroll to load products
for scroll in range(6):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for x in soup.find_all('div', {'class': 'css-1sn1xa2'}):
    try:
        # Get product name and price
        name = x.find('div', {'class': 'prd_link-product-name css-3um8ox'})
        price = x.find('div', {'class': 'prd_link-product-price css-h66vau'})
        rating_overall = x.find('span', {'class' : 'prd_rating-average-text css-t70v7i'})
        sold = x.find('span', {'class': 'prd_label-integrity css-1sgek4h'})

        names.append(name.get_text() if name else None)
        prices.append(price.get_text() if price else None)
        ratings_overall.append(rating_overall.get_text() if rating_overall else None)
        solds.append(sold.get_text() if sold else None)

    except Exception as e:
        print(f"Error: {e}")
        continue

driver.quit()
print('Scraping complete.')

print(f'length of names : {len(names)}')
print(f'length of prices : {len(prices)}')
print(f'length of ratings overall : {len(ratings_overall)}')
print(f'length of solds : {len(solds)}')    


In [ ]:
brands, ram, rom= [], [], []
brands.extend(['infinix' for i in range(7)])
ram.extend(['0' for i in range(7)])
rom.extend(['0' for i in range(7)])

In [ ]:

def clean_analyst(data):
    data = data.dropna()
    data['price'] = [number[2:] for number in data.price]
    data['price'] = [number.replace('.', '') for number in data.price]
    data['sold'] = [number.replace('+ terjual', '') for number in data.sold]
    data['sold'] = [number.replace(' terjual', '') for number in data.sold]
    
    for number in data.sold:
        k = ['1rb', '2rb', '3rb', '4rb', '5rb', '6rb', '7rb', '8rb', '9rb', '10rb']
        if number in k:
            data['sold'] = data['sold'].replace(number, number[:1] + '000')
    return data

In [ ]:
data = pd.DataFrame({'name':names, 'price':prices, 'rating':ratings_overall, 'sold':solds, 'brand':brands, 'ram':ram, 'rom':rom})
data = clean_analyst(data)
data.head()

In [ ]:
extracted_ram = data['name'].str.extract(r'(\d+)/')[0]
extracted_rom = data['name'].str.extract(r'/(\d+)GB')[0]

data.loc[data['ram']=='0', 'ram'] = extracted_ram
data.loc[data['rom']=='0', 'rom'] = extracted_rom

data['ram'] = data['ram'].fillna(data['name'].str.extract(r'(\d+)GB')[0])
data['rom'] = data['rom'].fillna(data['name'].str.extract(r'/(\d+)\)')[0])

In [ ]:
df = pd.read_csv('combined.csv')
df.head()

In [ ]:
df.info()

In [ ]:
combined_data = pd.concat([df, data], ignore_index=True)
combined_data.info()

In [ ]:
combined_data.to_csv('analyst.csv', index=False)